In [16]:
!pip install torch
!pip install transformers
!pip install datasets

/bin/bash: pip: command not found
/bin/bash: pip: command not found
/bin/bash: pip: command not found
/bin/bash: pip: command not found


In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [4]:
# import json 

# with open('../French-Politic-Speech-Dataset/speechs_data.json', 'r') as f:
#     dataset = json.load(f)['speechs']
#     dataset = [d for d in dataset if d['kind'] == 'Texte intégral']


# with open('./speechs_1000.json', 'w+') as f:
#     for i, line in enumerate(dataset[:1000]):
#         title = line['title']
#         texte = line['text']
#         newdata = {'text': title + ' : ' + texte }
#         newline = json.dumps(newdata, ensure_ascii=False) + '\n'
#         f.write(newline)

In [71]:
from datasets import load_dataset

dataset = load_dataset('json', data_files='./speechs.json')

Using custom data configuration default-953154b5fbdebce0


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 365.90it/s]


Dataset json downloaded and prepared to /Users/naowak/.cache/huggingface/datasets/json/default-953154b5fbdebce0/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00,  3.46it/s]


In [ ]:
from transformers import GPTJForCausalLM, AutoTokenizer

model = GPTJForCausalLM.from_pretrained("gustavecortal/fr-boris-8bit")
tokenizer = AutoTokenizer.from_pretrained("Cedille/fr-boris")

In [ ]:
# With transformers

prompt = (
    "Le président Pompidou était "
)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids,
    no_repeat_ngram_size=2, 
    do_sample=True,
    temperature=0.5,
    top_k=25,
    top_p=0.5,
    max_length=200,
)

gen_text = tokenizer.batch_decode(gen_tokens)[0]
print(gen_text)

In [72]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = dataset.map(tokenize_function, batched=True).shuffle(42)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 95671
    })
})

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="data/model_gptj_overton",
    num_train_epochs=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()